In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Serving Open Models on Vertex AI using vLLM with CPU

<table align="left">
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/prediction/vertexai_serving_vllm/vertexai_serving_vllm_cpu_llama3_2_3B.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fprediction%2Fvertexai_serving_vllm%2Fvertexai_serving_vllm_cpu_llama3_2_3B.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/vertexai_serving_vllm/vertexai_serving_vllm_cpu_llama3_2_3B.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

There are multiple ways of serving open models (including open source and open weight) such as Llama 3.2 on Google Cloud Vertex AI. The Llama models are available in [Model Garden](https://cloud.google.com/vertex-ai/generative-ai/docs/partner-models/llama) and Model Garden allows a single click self-deployment of the models. This notebooks demonstrates how Llama 3.2 3B model can be served via Vertex AI Endpoint using a custom vLLM container image built for the CPU. This notebook does the following:

- Builds a custom docker container image using vLLM source code
- Uploads the model to Model Registry using custom docker container image
- Creates a public Endpoint for Online Prediction
- Deploys model to the Endpoint
- Llama 3.2 3B model is downloaded from Hugging Face during deployment
- This custom container image can also be used for downloading model from Google Storage

The code in this notebook can be used for serving other open models supported by vLLM. This notebook has been tested with Python 3.10 and `google-cloud-aiplatform` SDK Version `1.106.0`.

To download the models from the Hugging Face, you need a Hugging Face token.
  1.  Create a [Hugging Face account](https://huggingface.co/) if you don't have one.
  2.  For **gated models** like Llama 3.2, ensure you have requested and been granted access on Hugging Face before proceeding.
  3.  Generate an Access Token: Go to **Your Profile > Settings > Access Tokens**.
  4.  Select **New Token**.
  5.  Specify a Name and a Role of at least Read.
  6.  Select **Generate a token**.
  7.  Set the token in hf_token env below.

## Get Started

### Install Vertex AI SDK for Python and other required packages

In [ ]:
!pip install --upgrade --quiet google-cloud-aiplatform

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project and initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
DEVICE_TYPE = "cpu"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

Set Project ID in active gcloud configuration.

In [ ]:
! gcloud config set project {PROJECT_ID}

## Create vLLM Customer Container Image for Vertex AI

The vLLM custom container image requires gcloud SDK so that it can download the model from Google Cloud Storage when Hugging Face token is not provided. The `entrypoint` script has also been updated to enable this feature.

### Enable Artifact Registry API
Enable the Artifact Registry API service for the Google cloud project. This tutorial requires [gcloud CLI](https://cloud.google.com/sdk/docs/install) installed.

In [ ]:
! gcloud components update --quiet && gcloud services enable artifactregistry.googleapis.com

### Create a private Docker repository
Create a Docker repository in [Artifact Registry](https://cloud.google.com/artifact-registry/docs/overview).

In [ ]:
DOCKER_REPOSITORY = "my-docker-repo"
! gcloud artifacts repositories create {DOCKER_REPOSITORY} --repository-format=docker --location={LOCATION} --description="Vertex AI Docker repository"

### Build vLLM Custom Docker Container Image for CPU

Clone vertex-ai-samples code reposistory.

In [ ]:
! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

Build image using Cloud Build

In [ ]:
! cd vertex-ai-samples/notebooks/official/prediction/vertexai_serving_vllm/cloud-build \
    && gcloud builds submit --config=cloudbuild.yaml --region={LOCATION} --timeout "2h" --machine-type=e2-highcpu-32 --substitutions=_REPOSITORY={DOCKER_REPOSITORY},_DEVICE_TYPE={DEVICE_TYPE},_BASE_IMAGE=vllm-cpu-base

## Deploy Model to Vertex AI Endpoint

Following steps are required to serve model via a Vertex AI Prediction Endpoint:
- import model to model registry
- create a Online Prediction Endpoint
- Deploy the model to endpoint

### Define Variable

In [ ]:
hf_token = "[your-hugging-face-auth-token]"  # @param {type:"string"}
model_name = "cpu-llama3_2_3B-serve-vllm"  # @param {type:"string"}
model_id = "meta-llama/Llama-3.2-3B"  # @param {type:"string"}
machine_type = "c2-standard-16"  # @param {type:"string"}
DOCKER_URI = (
    f"{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{DOCKER_REPOSITORY}/vllm-{DEVICE_TYPE}"
)

### Import model to Model Registry

In [ ]:
from google.cloud import aiplatform


def upload_model(
    model_name: str,
    model_id: str,
    hf_token: str,
    docker_uri: str,
) -> aiplatform.Model:

    vllm_args = [
        "python3",
        "-m",
        "vllm.entrypoints.openai.api_server",
        "--host=0.0.0.0",
        "--port=8080",
        f"--model={model_id}",
        "--max-model-len=2048",
    ]

    env_vars = {
        "HF_TOKEN": hf_token,
    }

    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=docker_uri,
        serving_container_args=vllm_args,
        serving_container_ports=[8080],
        serving_container_predict_route="/v1/completions",
        serving_container_health_route="/health",
        serving_container_environment_variables=env_vars,
        serving_container_shared_memory_size_mb=(16 * 1024),  # 16 GB
        serving_container_deployment_timeout=1800,
    )
    return model


vertexai_model = upload_model(
    model_name=model_name, model_id=model_id, hf_token=hf_token, docker_uri=DOCKER_URI
)

### Create Vertex AI Endpoint for Online Prediction

In [ ]:
def create_model_endpoint(model_name: str) -> aiplatform.Endpoint:
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    return endpoint


vertexai_endpoint = create_model_endpoint(model_name=model_name)

### Deploy Model to Endpoint
**NOTE**: The model deployment may take around 30 minutes to complete.

In [ ]:
def deploy_model(
    model: aiplatform.Model,
    endpoint: aiplatform.Endpoint,
    model_name: str,
    machine_type: str,
):
    print(
        f"Deploying {model_name} to endpoint: {endpoint.resource_name} using machine type: {machine_type}"
    )
    model.deploy(
        endpoint=endpoint,
        deployed_model_display_name=model_name,
        machine_type=machine_type,
        min_replica_count=1,
        max_replica_count=4,
        autoscaling_target_cpu_utilization=60,
        traffic_percentage=100,
        deploy_request_timeout=1800,
    )


deploy_model(
    model=vertexai_model,
    endpoint=vertexai_endpoint,
    model_name=model_name,
    machine_type=machine_type,
)

## Test Endpoint

In [ ]:
import json

PROMPT = "Distance of moon from earth is "
request_body = json.dumps(
    {
        "prompt": PROMPT,
        "temperature": 0.0,
    },
)

raw_response = vertexai_endpoint.raw_predict(
    body=request_body, headers={"Content-Type": "application/json"}
)
assert raw_response.status_code == 200
result = json.loads(raw_response.text)

for choice in result["choices"]:
    print(choice)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, delete the resources created in this tutorial.

### Delete Vertex AI Prediction Endpoint

In [ ]:
vertexai_endpoint.delete(force=True, sync=True)

### Delete Model

In [ ]:
vertexai_model.delete(sync=True)

### Delete private docker repository

In [ ]:
! gcloud artifacts repositories delete {DOCKER_REPOSITORY} --location={LOCATION} --quiet